In [4]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.figure_factory as ff
import plotly.graph_objs as go
from scipy.ndimage import gaussian_filter1d
import os
# 指定文件夹路径
folder_path = './air_quality'

# 获取文件夹中的所有文件名
filenames = os.listdir(folder_path)

# 读取所有 CSV 文件
Air_Qualities = [pd.read_csv(os.path.join(folder_path, filename)) for filename in filenames if filename.endswith('.csv')]


,No,year,month,day,hour,PM2.5,PM10,SO2,NO2,CO,O3,TEMP,PRES,DEWP,RAIN,wd,WSPM,station
0,1,2013,3,1,0,4.0,4.0,4.0,7.0,300.0,77.0,-0.7,1023.0,-18.8,0.0,NNW,4.4,Aotizhongxin
1,2,2013,3,1,1,8.0,8.0,4.0,7.0,300.0,77.0,-1.1,1023.2,-18.2,0.0,N,4.7,Aotizhongxin
2,3,2013,3,1,2,7.0,7.0,5.0,10.0,300.0,73.0,-1.1,1023.5,-18.2,0.0,NNW,5.6,Aotizhongxin
3,4,2013,3,1,3,6.0,6.0,11.0,11.0,300.0,72.0,-1.4,1024.5,-19.4,0.0,NW,3.1,Aotizhongxin
4,5,2013,3,1,4,3.0,3.0,12.0,12.0,300.0,72.0,-2.0,1025.2,-19.5,0.0,N,2.0,Aotizhongxin


In [ ]:
def simulate_pollutant_euler(t0, P0, delta_t, delta_P, wind_speed, D, simulation_time):
    """
    使用欧拉方法模拟污染物随时间的变化。

    参数：
    t0 -- 初始时间
    P0 -- 初始污染物量
    delta_t -- 时间步长
    delta_P -- 污染物增量
    wind_speed -- 风速
    D -- 距离
    simulation_time -- 模拟时间

    返回：
    times -- 时间列表
    pollutants -- 污染物量列表
    """
    times = [t0]
    pollutants = [P0]

    while times[-1] < simulation_time:
        # 计算实际移动距离
        actual_distance = D * wind_speed

        # 计算新的污染物量
        P1 = pollutants[-1] + delta_P

        # 更新时间
        t1 = times[-1] + delta_t

        # 更新污染物量
        pollutants.append(P1)
        times.append(t1)

    return times, pollutants